# Extracting data with the BigQuery API

In [1]:
import pandas as pd
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from google.cloud import bigquery
#%load_ext google.cloud.bigquery

In [2]:
SERVICE_ACCOUNT= 'service-acc1-0682'
JSON_FILE_NAME = 'agile-coral-324121-9a2fcd9c0a8f.json'
GCP_PROJECT_ID = 'agile-coral-324121'

In [3]:
client = bigquery.Client.from_service_account_json(JSON_FILE_NAME)
dataset_ref = client.dataset('hacker_news', project='bigquery-public-data')
dataset = client.get_dataset(dataset_ref)

In [4]:
full_dataset_id = "{}.{}".format(dataset.project, dataset.dataset_id)
friendly_name = dataset.friendly_name

print(dataset.description)

This dataset contains all stories and comments from Hacker News from its launch in 2006.  Each story contains a story id, the author that made the post, when it was written, and the number of points the story received.


In [5]:
data_tables = [x.table_id for x in client.list_tables(dataset)]
print(data_tables)

['comments', 'full', 'full_201510', 'stories']


In [6]:
comments = client.get_table(dataset.table('comments'))
print(comments.num_rows)
display(pd.DataFrame(comments.schema))

8399417


,0
0,"SchemaField('id', 'INTEGER', 'NULLABLE', 'Uniq..."
1,"SchemaField('by', 'STRING', 'NULLABLE', 'Usern..."
2,"SchemaField('author', 'STRING', 'NULLABLE', 'U..."
3,"SchemaField('time', 'INTEGER', 'NULLABLE', 'Un..."
4,"SchemaField('time_ts', 'TIMESTAMP', 'NULLABLE'..."
5,"SchemaField('text', 'STRING', 'NULLABLE', 'Com..."
6,"SchemaField('parent', 'INTEGER', 'NULLABLE', '..."
7,"SchemaField('deleted', 'BOOLEAN', 'NULLABLE', ..."
8,"SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is..."
9,"SchemaField('ranking', 'INTEGER', 'NULLABLE', ..."


In [7]:
full = client.get_table(dataset.table('full'))
print(full.num_rows)
display(pd.DataFrame(comments.schema))

28218187


,0
0,"SchemaField('id', 'INTEGER', 'NULLABLE', 'Uniq..."
1,"SchemaField('by', 'STRING', 'NULLABLE', 'Usern..."
2,"SchemaField('author', 'STRING', 'NULLABLE', 'U..."
3,"SchemaField('time', 'INTEGER', 'NULLABLE', 'Un..."
4,"SchemaField('time_ts', 'TIMESTAMP', 'NULLABLE'..."
5,"SchemaField('text', 'STRING', 'NULLABLE', 'Com..."
6,"SchemaField('parent', 'INTEGER', 'NULLABLE', '..."
7,"SchemaField('deleted', 'BOOLEAN', 'NULLABLE', ..."
8,"SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is..."
9,"SchemaField('ranking', 'INTEGER', 'NULLABLE', ..."


In [8]:
full_201510 = client.get_table(dataset.table('full_201510'))
print(full_201510.num_rows)
display(pd.DataFrame(full_201510.schema))

18778427


,0
0,"SchemaField('by', 'STRING', 'NULLABLE', 'Usern..."
1,"SchemaField('score', 'INTEGER', 'NULLABLE', 'S..."
2,"SchemaField('time', 'INTEGER', 'NULLABLE', 'Un..."
3,"SchemaField('title', 'STRING', 'NULLABLE', 'St..."
4,"SchemaField('type', 'STRING', 'NULLABLE', 'Typ..."
5,"SchemaField('url', 'STRING', 'NULLABLE', 'Stor..."
6,"SchemaField('text', 'STRING', 'NULLABLE', 'Sto..."
7,"SchemaField('parent', 'INTEGER', 'NULLABLE', '..."
8,"SchemaField('deleted', 'BOOLEAN', 'NULLABLE', ..."
9,"SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is..."


In [9]:
stories = client.get_table(dataset.table('stories'))
print(stories.num_rows)
pd.DataFrame(stories.schema)

1959809


,0
0,"SchemaField('id', 'INTEGER', 'NULLABLE', 'Uniq..."
1,"SchemaField('by', 'STRING', 'NULLABLE', 'Usern..."
2,"SchemaField('score', 'INTEGER', 'NULLABLE', 'S..."
3,"SchemaField('time', 'INTEGER', 'NULLABLE', 'Un..."
4,"SchemaField('time_ts', 'TIMESTAMP', 'NULLABLE'..."
5,"SchemaField('title', 'STRING', 'NULLABLE', 'St..."
6,"SchemaField('url', 'STRING', 'NULLABLE', 'Stor..."
7,"SchemaField('text', 'STRING', 'NULLABLE', 'Sto..."
8,"SchemaField('deleted', 'BOOLEAN', 'NULLABLE', ..."
9,"SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is..."


In [10]:
query = (
    """SELECT * 
    FROM  `bigquery-public-data.hacker_news.full` 
    LIMIT 10000""")
   
first1000Full =client.query(query).result().to_dataframe()

In [11]:
first1000Full

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,Here&#x27;s a scenario. Someone sends you a me...,None,munin,NaN,1440892117,2015-08-29 23:48:37+00:00,comment,10141767,10141670,NaN,NaN,None
1,None,None,Every time &quot;electrosensitivity&quot; come...,None,userbinator,NaN,1440892023,2015-08-29 23:47:03+00:00,comment,10141766,10140844,NaN,NaN,None
2,None,None,You should check it out and decide for yoursel...,None,xanthor,NaN,1440891994,2015-08-29 23:46:34+00:00,comment,10141765,10141703,NaN,NaN,None
3,None,None,"&quot;,&quot; too",None,mistercow,NaN,1440891993,2015-08-29 23:46:33+00:00,comment,10141764,10141152,NaN,NaN,None
4,None,None,Just a bit of feedback (bias: I&#x27;m used to...,None,disposablehero,NaN,1440891992,2015-08-29 23:46:32+00:00,comment,10141763,10140728,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,None,None,The author&#x27;s primary motivating example i...,None,mcphage,NaN,1439571939,2015-08-14 17:05:39+00:00,comment,10061279,10060029,NaN,NaN,None
9996,None,None,So Apple should be forced to develop for a com...,None,scarface74,NaN,1585700421,2020-04-01 00:20:21+00:00,comment,22744319,22744312,NaN,NaN,None
9997,None,None,I have always played bots with a handicap rath...,None,kryptiskt,NaN,1418038183,2014-12-08 11:29:43+00:00,comment,8715939,8715558,NaN,NaN,None
9998,None,None,scoped monkey would be quite usefull? One coul...,None,aikah,NaN,1402780199,2014-06-14 21:09:59+00:00,comment,7893703,7893520,NaN,NaN,None
